# Custom model example

In [ ]:
from sklearn import datasets
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# load datasets
digits = datasets.load_digits()
X = digits.data
y = digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# model training
model = XGBClassifier()
model.fit(X_train, y_train)

# predict score
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# Save model artifact to local filesystem (doesn't persist)
artifact_filename = 'model.bst'
local_path = artifact_filename
model.save_model(local_path)

# Upload model to GCS

In [ ]:
bucket_name = 'cathay-test'
gcs = 'gs://' + bucket_name

## create bucket

In [ ]:
# !gsutil mb -l us-central1 $bucket_name

## Upload model artifact to Cloud Storage

In [ ]:
from google.cloud import storage
import os

model_directory = f'{gcs}/' + 'models' + '/' + 'xgboost'
storage_path = os.path.join(model_directory, artifact_filename)
blob = storage.blob.Blob.from_string(storage_path, client=storage.Client())
blob.upload_from_filename(local_path)

# Upload model to Vertex AI Platform

In [ ]:
project = 'cathay-test'
location = 'us-central1'
display_name = 'digits-xgboost-v1'
serving_container_image_uri = 'us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-4:latest'
artifact_uri = model_directory

In [ ]:
from utils import upload_model_sample

model = upload_model_sample.upload_model_sample(
    project=project,
    location=location,
    display_name=display_name,
    serving_container_image_uri=serving_container_image_uri,
    artifact_uri=artifact_uri
)
model

In [ ]:
deployment = model.deploy(
    deployed_model_display_name = 'serving-digits-xgboost-v1',
    machine_type = 'n1-standard-2',
)
deployment